In [ ]:
import asyncio
from rtlsdr import RtlSdr

import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

import numpy as np
from scipy import signal

import nest_asyncio
nest_asyncio.apply()

network = input_data(shape=[None, 32, 32, 1])

network = conv_2d(network, 128, 5, activation='prelu')
network = conv_2d(network, 96, 1, activation='prelu')
network = conv_2d(network, 64, 1, activation='prelu')
network = max_pool_2d(network, 3, strides=2)
network = conv_2d(network, 128, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = conv_2d(network, 128, 5, activation='prelu')
network = conv_2d(network, 128, 1, activation='prelu')
network = conv_2d(network, 128, 1, activation='prelu')
network = avg_pool_2d(network, 3, strides=2)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = fully_connected(network, 64, activation='tanh')
network = fully_connected(network, 4, activation='softmax')

model = tflearn.DNN(network, tensorboard_dir='tflearn_logs')
model.load('radio-identification20220412-093855.tfl')        


async def streaming():
    sdr = RtlSdr()
    N = 256*332+4096   # number of samples
    sdr.sample_rate = 2.4e6
    sdr.center_freq = 433.9e6
    sdr.gain = 4

    async for samples in sdr.stream():
        # do something with samples
        # ...
                    
        samples = sdr.read_samples(N)
        # separating real and imaginary parts of the signal
        real = np.real(samples)
        imag = np.imag(samples)

        # getting the amplitude
        amplitude = real.copy()
        for n in range(len(amplitude)):
            amplitude[n] = (real[n]**2+imag[n]**2)**(1/2)
            n = n + 1
        
        # check signal reception 
        if np.mean(amplitude[4096:]) < 0.1:
            print("no signal")
        else:
            # Signal Processing
            # 1: filtering
            b, a = signal.butter(3, 0.3) # order 3 lowpass butterworth filter
            filtered_signal = signal.filtfilt(b, a, amplitude)

            # 2: normalization 0-1
            normalized_signal = (filtered_signal[4096:]-np.min(filtered_signal[4096:]))/np.ptp(filtered_signal[4096:])
            
            # reshape
            normalized_signal = normalized_signal.reshape(-1,32,32,1)
            
            # prediction
            predictions = model.predict(normalized_signal)
            #print(predictions)
            radio1 = 0
            radio2 = 0
            radio3 = 0
            radio4 = 0

            for predict in range(len(predictions)):
                lista = [predictions[predict,0],predictions[predict,1],predictions[predict,2],predictions[predict,3]]
                radio = lista.index(max(lista))+1
                if radio == 1: radio1 = radio1+1
                if radio == 2: radio2 = radio2+1
                if radio == 3: radio3 = radio3+1 
                if radio == 4: radio4 = radio4+1

            lista = [radio1,radio2,radio3,radio4]
            print("radio"+str(lista.index(max(lista))+1))

    # to stop streaming:
    await sdr.stop()

    # done
    sdr.close()

loop = asyncio.get_event_loop()
loop.run_until_complete(streaming())